In [1]:
x1 = rand(2)

2-element Array{Float64,1}:
 0.290939
 0.256881

In [14]:
n=10^2;
pi_mc = 0;
for ii=1:n
    x1 = rand(2);
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
println(4*pi_mc/n)

3.52


In [15]:
n=10^3;
pi_mc = 0;
for ii=1:n
    x1 = rand(2);
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
println(4*pi_mc/n)

3.092


In [16]:
n=10^4;
pi_mc = 0;
for ii=1:n
    x1 = rand(2);
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
println(4*pi_mc/n)

3.1532


In [17]:
n=10^5;
pi_mc = 0;
for ii=1:n
    x1 = rand(2);
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
println(4*pi_mc/n)

3.1438


In [13]:
n=10^6;
pi_mc = 0;
for ii=1:n
    x1 = rand(2);
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
println(4*pi_mc/n)

3.140904


In [18]:
n=10^7;
pi_mc = 0;
for ii=1:n
    x1 = rand(2);
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
println(4*pi_mc/n)

3.1420144


In [19]:
n=10^8;
pi_mc = 0;
for ii=1:n
    x1 = rand(2);
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
println(4*pi_mc/n)

3.14169592


In [96]:
n=10^4;
pi_mc = 0;
for nn=1:n
for ii=1:10
    x1 = rand(2);
    x1[1] = (x1[1] + (ii-1))*0.1;
    if(norm(x1,2)<1);
        pi_mc = pi_mc+1;
    end
end
end
println(4*pi_mc/(10*n))

3.14052


In [87]:
rand(2)

2-element Array{Float64,1}:
 0.123846
 0.330859

In [49]:
n=100;
pi_mc = 0;
for nn=1:n
for ii=1:10
    x1 = rand(2);
    for jj=1:10
        x1[1] = (x1[1] + (ii-1))*0.1;
        x1[2] = (x1[2] + (jj-1))*0.1;
        if(norm(x1,2)<1);
            pi_mc = pi_mc+1;
        end
    end
end
end
println(4*pi_mc/(100*n))

2.9788


In [50]:
n=1000;
pi_mc = 0;
for nn=1:n
for ii=1:10
    x1 = rand(2);
    for jj=1:10
        x1[1] = (x1[1] + (ii-1))*0.1;
        x1[2] = (x1[2] + (jj-1))*0.1;
        if(norm(x1,2)<1);
            pi_mc = pi_mc+1;
        end
    end
end
end
println(4*pi_mc/(100*n))

2.97728
